In [3]:
import pandas as pd
import re

In [4]:
list_csv = ['data/kor/attraction_review',
            'data/kor/hotel_review',
            'data/kor/restaurant_review',
            'data/eng/eng_attraction_review',
            'data/eng/eng_hotel_review',
            'data/eng/eng_restaurant_review']

In [57]:
for csv in list_csv:
    df = pd.read_csv(csv+'.csv',
                         names=['placeId','userId','score','title','review'],
                         encoding='euc-kr')
    print('na',len(df[df.userId.isna()]),'개 ',end='')
    count = 0
    for i in df.userId:
        if not pd.isna(i) and (re.fullmatch(r' +', i) != None):
            count+=1
    print('공백', count,'개')

na 0 개 공백 25 개
na 0 개 공백 2 개
na 7120 개 공백 0 개
na 934 개 공백 0 개
na 0 개 공백 4 개
na 2730 개 공백 0 개


In [5]:
def unicode(match):
        match = match.group()
        return chr(int(match[2:], base=16))

In [8]:
##For user-based
for csv in list_csv:
    df_raw = pd.read_csv(csv+'.csv',
                         names=['placeId','userId','score','title','review'],
                         encoding='ms949')
    df_raw = df_raw.drop(df_raw.columns[5:],axis=1)
#drop rows with NaN in user
    before = len(df_raw)
    df_raw = df_raw.dropna()
    df_raw = df_raw.drop(df_raw.columns[0], axis=1)
    df_raw.review = df_raw.review.apply(lambda x: re.sub('(<br/>|[ ]+)',' ',x))
    df_raw = df_raw[df_raw.userId.str.match(' ')==False]
    after = len(df_raw)
    print(before - after, 'entries droped in ', csv)
    df_raw = df_raw.sort_values(by=['userId']) 
# 
    df_raw.score = df_raw.score/10
    df_raw.score = df_raw.score.apply(int)
#merge title into review
    df_raw.review = df_raw.title + ' ' + df_raw.review
    df_raw = df_raw.drop(columns=['title'])
# unicode replace #
    
    for i in df_raw.userId:
        i = re.sub(r'\\u[0-9a-f]{4}', unicode, i)
    df_raw.userId = df_raw.userId.apply(lambda x:
    re.sub(r'\\u[0-9a-f]{4}', unicode, x))
###################
    df = df_raw.reset_index(drop=True)
    df.to_csv(csv+'_user.csv',mode='w')

36 entries droped in  data/kor/attraction_review
3 entries droped in  data/kor/hotel_review
20828 entries droped in  data/kor/restaurant_review
1319 entries droped in  data/eng/eng_attraction_review
815 entries droped in  data/eng/eng_hotel_review
21491 entries droped in  data/eng/eng_restaurant_review


In [8]:
##For tag-based
for csv in list_csv:
    df_raw = pd.read_csv(csv+'.csv',
                         names=['placeId','userId','score','title','review'],
                         encoding='ms949')
    df_raw = df_raw.drop(df_raw.columns[5:],axis=1)
#drop rows with NaN in user
    before = len(df_raw)
    df_raw = df_raw.dropna()
    df_raw = df_raw.drop(df_raw.columns[0], axis=1)
    df_raw.review = df_raw.review.apply(lambda x: re.sub('(<br/>|[ ]+)',' ',x))
    df_raw = df_raw[df_raw.userId.str.match(' ')==False]
    after = len(df_raw)
    print(before - after, 'entries droped in ', csv)
    df_raw = df_raw.sort_values(by=['userId']) 
# 
    df_raw.score = df_raw.score/10
    df_raw.score = df_raw.score.apply(int)
#merge title into review
    df_raw.review = df_raw.title + ' ' + df_raw.review
    df_raw = df_raw.drop(columns=['title'])
# unicode replace #
    
    for i in df_raw.userId:
        i = re.sub(r'\\u[0-9a-f]{4}', unicode, i)
    df_raw.userId = df_raw.userId.apply(lambda x:
    re.sub(r'\\u[0-9a-f]{4}', unicode, x))
###################
    df = df_raw.reset_index(drop=True)
    df.to_csv(csv+'_tag.csv',mode='w')

36 entries droped in  data/kor/attraction_review
3 entries droped in  data/kor/hotel_review
20828 entries droped in  data/kor/restaurant_review
1319 entries droped in  data/eng/eng_attraction_review
815 entries droped in  data/eng/eng_hotel_review
21491 entries droped in  data/eng/eng_restaurant_review
